In [ ]:
import tensorflow as tf
import csv
from numpy import asarray, save
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import time

# load data separated into training and test set dataset
(ds_train, ds_test), ds_info = tfds.load('stanford_dogs', split=['train', 'test'], shuffle_files=True, with_info=True)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Resize the images to fit the input for AlexNet (227x227)

#____________________________________________________________________________________________________________________________
# Resize the images to fit the input for AlexNet (227x227)

# Training Set
train_lst_imgs = []
train_lst_labels = []

print("The images will be resized now.")

kaggle_breeds = ('n02105162-malinois', 'n02094258-Norwich_terrier', 'n02102177-Welsh_springer_spaniel', 'n02086646-Blenheim_spaniel', 'n02086910-papillon', 'n02093256-Staffordshire_bullterrier', 'n02113624-toy_poodle', 'n02105056-groenendael', 'n02109961-Eskimo_dog', 'n02116738-African_hunting_dog', 'n02096177-cairn', 'n02096585-Boston_bull', 'n02100735-English_setter', 'n02102973-Irish_water_spaniel', 'n02099429-curly-coated_retriever', 'n02088364-beagle', 'n02101006-Gordon_setter', 'n02108089-boxer', 'n02097130-giant_schnauzer', 'n02112137-chow')

for train in tfds.as_numpy(ds_train):  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
    #print(list(example.keys()))

    #resize the image to 227 x 227 because this is what AlexNet uses
    #if you want it with padding:
    #train_lst_imgs.append(tf.image.resize_with_pad(train["image"], target_height=227, target_width=227))

    #Do a subset of the first 20 dog breeds to make it run faster

    if train["label"] <= 20:
        train_lst_imgs.append(np.array(tf.image.resize(train["image"], (227, 227))))
        label = train["label"]
        train_lst_labels.append(label)

# Test Set
test_lst_imgs = []
test_lst_labels = []

for test in tfds.as_numpy(ds_test):  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
    #print(list(example.keys()))

    #resize the image to 227 x 227 because this is what AlexNet uses

    #Do a subset of the first 20 dog breeds to make it run faster
    if test["label"] <= 20:
        test_lst_imgs.append(np.array(tf.image.resize(test["image"], (227, 227))))
        label = test["label"]
        test_lst_labels.append(label)



#verify that all of the images/labels were transferred when they were resized
print("The length of the training list of resized images is:", len(train_lst_imgs))
print("The length of the training list of labels is:", len(train_lst_labels))

print("The length of the test list of resized images is:", len(test_lst_imgs))
print("The length of the test list of labels is:", len(test_lst_labels))

print(type(train_lst_imgs),type(train_lst_labels))
#____________________________________________________________________________________________________________________________

# Convert into NumPy Arrays

# this can take a while - in TF2.X, apparently it's slow to convert list of tensors to numpy.array
x_train = np.array(train_lst_imgs) #images
y_train = np.array(train_lst_labels) #labels


x_test = np.array(test_lst_imgs) #images
y_test = np.array(test_lst_labels) #labels

print("The images shape after converting into NumPy arrays is: ",x_train.shape,"\nThe labels shape after converting into NumPy arrays is: ",y_train.shape)
print("Their types are: ")
print(type(x_train),type(y_train))


#____________________________________________________________________________________________________________________________


# Normalize the data

# Training Set
xn_train = x_train.astype(np.float32)
yn_train = y_train.astype(np.int32)
#xn_train = xn_train/255
print("Training set images shape after normalization = ", xn_train.shape)


# Test Set
xn_test = x_test.astype(np.float32)
yn_test = y_test.astype(np.int32)
print("Test set images shape after normalization = ", xn_test.shape)



#____________________________________________________________________________________________________________________________


print("The model will now be defined and compiled.")

The images will be resized now.
The length of the training list of resized images is: 2100
The length of the training list of labels is: 2100
The length of the test list of resized images is: 1715
The length of the test list of labels is: 1715
<class 'list'> <class 'list'>
The images shape after converting into NumPy arrays is:  (2100, 227, 227, 3) 
The labels shape after converting into NumPy arrays is:  (2100,)
Their types are: 
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
Training set images shape after normalization =  (2100, 227, 227, 3)
Test set images shape after normalization =  (1715, 227, 227, 3)
The model will now be defined and compiled.


In [ ]:
# Define the model

# Import a few specific libraries/layers
from tensorflow import keras
from tensorflow.keras import layers, models

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout

model=Sequential()

#1 conv layer
model.add(Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),padding="valid",activation="relu",input_shape=(227,227,3)))

#1 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())

#2 conv layer
model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding="valid",activation="relu"))

#2 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())

#3 conv layer
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))

#4 conv layer
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))

#5 conv layer
model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))

#3 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())


model.add(Flatten())

#1 dense layer
model.add(Dense(4096,input_shape=(227,227,3),activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#2 dense layer
model.add(Dense(4096,activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#3 dense layer
model.add(Dense(1000,activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#output layer
model.add(Dense(120,activation="softmax"))

model.summary()

#____________________________________________________________________________________________________________________________

# Compile the model

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

print("Model is compiled.")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 27, 27, 96)        0         
 g2D)                                                            
                                                                 
 batch_normalization_6 (Bat  (None, 27, 27, 96)        384       
 chNormalization)                                                
                                                                 
 conv2d_6 (Conv2D)           (None, 23, 23, 256)       614656    
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 11, 11, 256)       0         
 g2D)                                                            
                                                      

In [ ]:

print("Now the model will be trained. This may take around 20-30 minutes depending on image class size and epoch size.")
# Train the model

start = time.time()
history = model.fit(xn_train, yn_train, epochs=100)
end = time.time()
total_time = end-start

#____________________________________________________________________________________________________________________________

# Convert the time from seconds

def convert(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))

total_time = convert(total_time)

#____________________________________________________________________________________________________________________________

# Save the epoch accuracy data

with open('output.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(model.history.history['accuracy'])

#____________________________________________________________________________________________________________________________

# Evaluate the loss and accuracy

la = []

loss, accuracy = model.evaluate(xn_test, yn_test)

print("Training Runtime: ", total_time)
print("The Test Set Loss is: ", loss, "\nThe Test Set Accuracy is: ", accuracy)

la.append(loss)
la.append(accuracy)
la.append(total_time)


#____________________________________________________________________________________________________________________________

# Predict values with the model

pred = model.predict(xn_test)

#____________________________________________________________________________________________________________________________

# Download xn_test and yn_test to create images of the predicted and actual dog breed estimates

#randomly select 9 images

images = []
actual = []
pred_label = []

for i in range(9):
    r = np.random.randint( 0, xn_test.shape[0], 1)
    images.append(xn_test[r[0]])
    actual.append(yn_test[r[0]])
    pred_label.append(yn_test[r[0]]*pred[r[0]][yn_test[r[0]]])

save('images.npy', images)

with open('pred_labels.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(pred_label)

with open('actual_labels.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(actual)

with open('test.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(la)


Now the model will be trained. This may take around 20-30 minutes depending on image class size and epoch size.
Epoch 1/100
66/66 [==============================] - 54s 775ms/step - loss: 4.7287 - accuracy: 0.0857
Epoch 2/100
66/66 [==============================] - 51s 770ms/step - loss: 3.9037 - accuracy: 0.1048
Epoch 3/100
66/66 [==============================] - 51s 770ms/step - loss: 3.4686 - accuracy: 0.1162
Epoch 4/100
66/66 [==============================] - 51s 775ms/step - loss: 3.2248 - accuracy: 0.1295
Epoch 5/100
66/66 [==============================] - 52s 783ms/step - loss: 3.0728 - accuracy: 0.1495
Epoch 6/100
66/66 [==============================] - 51s 776ms/step - loss: 2.9496 - accuracy: 0.1519
Epoch 7/100
66/66 [==============================] - 51s 775ms/step - loss: 2.9181 - accuracy: 0.1724
Epoch 8/100
66/66 [==============================] - 51s 779ms/step - loss: 2.7766 - accuracy: 0.1867
Epoch 9/100
66/66 [==============================] - 51s 774ms/step - lo

In [ ]:
# save model to local path

model.save('/content/drive/MyDrive/AIStuff/myymodel')

In [ ]:
!pip install pyngrok==4.2.2
!pip install tensorflow
!pip install numpy
!pip install Pillow
!pip install requests
!pip install -U ipykernel
!pip install -q streamlit

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.2.2-py3-none-any.whl size=18016 sha256=e80d3032836a01254443d55199881a346f9a6e3268a93145fb0acde44d9b9508
  Stored in directory: /root/.cache/pip/wheels/04/ae/38/d5c5d22cec6ab0169b667c91ae35c400e961cc25e382a4c3ce
Successfully built pyngrok
  Attempting uninstall: pyngrok
    Found existing installation: pyngrok 7.0.3
    Uninstalling pyngrok-7.0.3:
      Successfully uninstalled pyngrok-7.0.3


In [ ]:
%%writefile score.py

# script that sets up ngrok app's user interface through streamline

import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image
import requests
from io import BytesIO
#hola
st.title("Dog Image Classifier")
st.text("Provide URL of dog Image for image classification")

def load_model():
    model = tf.keras.models.load_model('/content/drive/MyDrive/AIStuff/myymodel')
    return model

with st.spinner('Loading Model Into Memory...'):
  model = load_model()

classes=['n02085620-chihuahua', 'n02085782-japanese_spaniel', 'n02085936-maltese_dog', 'n02086079-pekinese', 'n02086240-shih-tzu', 'n02086646-blenheim_spaniel', 'n02086910-papillon', 'n02087046-toy_terrier', 'n02087394-rhodesian_ridgeback', 'n02088094-afghan_hound', 'n02088238-basset', 'n02088364-beagle', 'n02088466-bloodhound', 'n02088632-bluetick', 'n02089078-black-and-tan_coonhound', 'n02089867-walker_hound', 'n02089973-english_foxhound', 'n02090379-redbone', 'n02090622-borzoi', 'n02090721-irish_wolfhound', 'n02091032-italian_greyhound', 'n02091134-whippet', 'n02091244-ibizan_hound', 'n02091467-norwegian_elkhound', 'n02091635-otterhound', 'n02091831-saluki', 'n02092002-scottish_deerhound', 'n02092339-weimaraner', 'n02093256-staffordshire_bullterrier', 'n02093428-american_staffordshire_terrier', 'n02093647-bedlington_terrier', 'n02093754-border_terrier', 'n02093859-kerry_blue_terrier', 'n02093991-irish_terrier', 'n02094114-norfolk_terrier', 'n02094258-norwich_terrier', 'n02094433-yorkshire_terrier', 'n02095314-wire-haired_fox_terrier', 'n02095570-lakeland_terrier', 'n02095889-sealyham_terrier', 'n02096051-airedale', 'n02096177-cairn', 'n02096294-australian_terrier', 'n02096437-dandie_dinmont', 'n02096585-boston_bull', 'n02097047-miniature_schnauzer', 'n02097130-giant_schnauzer', 'n02097209-standard_schnauzer', 'n02097298-scotch_terrier', 'n02097474-tibetan_terrier', 'n02097658-silky_terrier', 'n02098105-soft-coated_wheaten_terrier', 'n02098286-west_highland_white_terrier', 'n02098413-lhasa', 'n02099267-flat-coated_retriever', 'n02099429-curly-coated_retriever', 'n02099601-golden_retriever', 'n02099712-labrador_retriever', 'n02099849-chesapeake_bay_retriever', 'n02100236-german_short-haired_pointer', 'n02100583-vizsla', 'n02100735-english_setter', 'n02100877-irish_setter', 'n02101006-gordon_setter', 'n02101388-brittany_spaniel', 'n02101556-clumber', 'n02102040-english_springer', 'n02102177-welsh_springer_spaniel', 'n02102318-cocker_spaniel', 'n02102480-sussex_spaniel', 'n02102973-irish_water_spaniel', 'n02104029-kuvasz', 'n02104365-schipperke', 'n02105056-groenendael', 'n02105162-malinois', 'n02105251-briard', 'n02105412-kelpie', 'n02105505-komondor', 'n02105641-old_english_sheepdog', 'n02105855-shetland_sheepdog', 'n02106030-collie', 'n02106166-border_collie', 'n02106382-bouvier_des_flandres', 'n02106550-rottweiler', 'n02106662-german_shepherd', 'n02107142-doberman', 'n02107312-miniature_pinscher', 'n02107574-greater_swiss_mountain_dog', 'n02107683-bernese_mountain_dog', 'n02107908-appenzeller', 'n02108000-entlebucher', 'n02108089-boxer', 'n02108422-bull_mastiff', 'n02108551-tibetan_mastiff', 'n02108915-french_bulldog', 'n02109047-great_dane', 'n02109525-saint_bernard', 'n02109961-eskimo_dog', 'n02110063-malamute', 'n02110185-siberian_husky', 'n02110627-affenpinscher', 'n02110806-basenji', 'n02110958-pug', 'n02111129-leonberg', 'n02111277-newfoundland', 'n02111500-great_pyrenees', 'n02111889-samoyed', 'n02112018-pomeranian', 'n02112137-chow', 'n02112350-keeshond', 'n02112706-brabancon_griffon', 'n02113023-pembroke', 'n02113186-cardigan', 'n02113624-toy_poodle', 'n02113712-miniature_poodle', 'n02113799-standard_poodle', 'n02113978-mexican_hairless', 'n02115641-dingo', 'n02115913-dhole', 'n02116738-african_hunting_dog']

def scale(image):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return tf.image.resize(image, [227, 227])

def decode_img(image):
    img = tf.image.decode_jpeg(image, channels=3)
    img = scale(img)
    return np.expand_dims(img, axis=0)

path = st.text_input('Enter Image URL to Classify..', 'https://www.akc.org/wp-content/uploads/2017/11/Chihuahua-standing-in-three-quarter-view.jpg')

if path is not None:
    content = requests.get(path).content  # Get image content
    image = decode_img(content)  # Process the image

    st.write("Predicted Class:")
    with st.spinner('classifying.....'):
        label = np.argmax(model.predict(image), axis=1)
        st.write(classes[label[0]])

    st.write("")
    image_display = Image.open(BytesIO(content))
    st.image(image_display, caption='Classifying Dog Image', use_column_width=True)

Writing score.py


In [ ]:
!nohup streamlit run score.py &

nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok

url = ngrok.connect(port=8501)
url

PyngrokNgrokHTTPError: ignored

In [ ]:
!cat nohup.out




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8548
  External URL: http://35.245.228.38:8548




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8549
  External URL: http://35.245.228.38:8549



In [ ]:
# Save Personal Auth Token from Ngrok user account
!ngrok authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import requests
import time

# Start time
start_time = time.time()

# Replace 'SERVER_ENDPOINT' and 'URL_TO_YOUR_IMAGE' with your server's endpoint and image URL
server_endpoint = 'https://8161-35-245-228-38.ngrok-free.app/'
image_url = 'https://cdn.britannica.com/16/234216-050-C66F8665/beagle-hound-dog.jpg'

# Send image to server
response = requests.post(server_endpoint, json={'image_url': image_url})

# End time
end_time = time.time()

# Calculate latency
latency = end_time - start_time
print(f"Latency: {latency} seconds")


Latency: 0.23293352127075195 seconds
